In [ ]:
pip install tensorflow

In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2021-07-05 17:20:16--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 307.92M  3.29MB/s    in 82s     

2021-07-05 17:21:39 (3.76 MB/s) - ‘/tmp/rockpaperscissors.zip’ saved [322873683/322873683]



In [3]:
# melakukan ekstraksi pada file zip
import zipfile,os
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/rockpaperscissors'
data_dir = os.path.join(base_dir, 'rps-cv-images')

In [4]:
pip install split-folders tqdm

In [5]:
import splitfolders

splitfolders.ratio(data_dir, output=data_dir+'/data', ratio=(.6, .4))

Copying files: 2188 files [00:00, 2404.21 files/s]


In [6]:
os.listdir(data_dir)

['paper', 'data', 'scissors', 'rock', 'README_rpc-cv-images.txt']

In [7]:
os.listdir(data_dir+'/data')

['val', 'train']

In [8]:
train_dir = os.path.join(data_dir, 'data/train')
val_dir = os.path.join(data_dir, 'data/val')

In [9]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

In [22]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(300, 200),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=8,
        # karena ini merupakan masalah klasifikasi 2 kelas maka menggunakan class_mode = 'binary'
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        val_dir, # direktori data validasi
        target_size=(300, 200), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=8, # karena ini merupakan masalah klasifikasi 2 kelas maka menggunakan class_mode = 'binary'
        class_mode='categorical')

Found 1312 images belonging to 3 classes.
Found 876 images belonging to 3 classes.


In [23]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(300, 200, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [24]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=20,
      epochs=25,
      validation_data=validation_generator,
      validation_steps=4,
      verbose=1)

Epoch 1/25
20/20 [==============================] - 39s 2s/step - loss: 1.3841 - accuracy: 0.3562 - val_loss: 1.0849 - val_accuracy: 0.4062
Epoch 2/25
20/20 [==============================] - 36s 2s/step - loss: 1.1384 - accuracy: 0.2937 - val_loss: 1.0763 - val_accuracy: 0.3750
Epoch 3/25
20/20 [==============================] - 36s 2s/step - loss: 1.1048 - accuracy: 0.3812 - val_loss: 1.0671 - val_accuracy: 0.4062
Epoch 4/25
20/20 [==============================] - 36s 2s/step - loss: 1.1190 - accuracy: 0.3438 - val_loss: 1.0360 - val_accuracy: 0.6562
Epoch 5/25
20/20 [==============================] - 36s 2s/step - loss: 0.9904 - accuracy: 0.5375 - val_loss: 0.9501 - val_accuracy: 0.4375
Epoch 6/25
20/20 [==============================] - 36s 2s/step - loss: 1.0602 - accuracy: 0.5938 - val_loss: 1.0810 - val_accuracy: 0.4375
Epoch 7/25
20/20 [==============================] - 35s 2s/step - loss: 0.7991 - accuracy: 0.6875 - val_loss: 0.6782 - val_accuracy: 0.7188
Epoch 8/25
20/20 [==